In [1]:
import torch
import joblib
import numpy as np

/data/dell/anaconda3/envs/torch1.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('torch_weights.txt', 'r') as f:
    lines = f.readlines()
tkeys = [iline.strip() for iline in lines]

In [3]:
with open('jax_weights.txt', 'r') as f:
    lines = f.readlines()
jkeys = [iline.strip() for iline in lines]

In [4]:
# directory for jax and pytorch weights
weight_dir = '/data/dell/Desktop/GitHub/ProteinMPNN_jax/weights/torch'
dest_dir = '/data/dell/Desktop/GitHub/ProteinMPNN_jax/weights/jax'

In [5]:
weight_list = ["v_48_010", "v_48_020"]

In [14]:
for iweight in weight_list:
    ck = torch.load(weight_dir + '/' + iweight + '.pt', map_location='cpu')
    ck_new = {}
    ck_new['num_edges'] = ck['num_edges']
    ck_new['noise_level'] = ck['noise_level']
    ck_new['model_state_dict'] = {}
    for ith, ijkey in enumerate(jkeys):
        itkey = tkeys[ith]
        if 'edge_embedding' in ijkey:
            ck_new['model_state_dict'][ijkey] = {'w': np.array(ck['model_state_dict'][itkey]).T}
        elif 'embed_token' in ijkey:
            ck_new['model_state_dict'][ijkey] = {'W_s': np.array(ck['model_state_dict'][itkey])}
        elif 'norm' in ijkey:
            ck_new['model_state_dict'][ijkey] = {'scale': np.array(ck['model_state_dict'][itkey + '.weight']),
                                                 'offset': np.array(ck['model_state_dict'][itkey + '.bias'])}
        else:
            ck_new['model_state_dict'][ijkey] = {'w': np.array(ck['model_state_dict'][itkey + '.weight']).T,
                                                 'b': np.array(ck['model_state_dict'][itkey + '.bias']).T}
    
    joblib.dump(ck_new, dest_dir + '/' + iweight + '.pkl')